In [13]:
from gensim.models import Word2Vec
from konlpy.tag import Okt
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv("./datasets/초등_사회_문항별_지문텍스트및메타.csv", index_col=0)
print(df.shape)

(13683, 12)


In [5]:
df = df.drop_duplicates(subset=['지문텍스트']).dropna(subset=['지문텍스트'])
print(df.shape)
df.head()

(12379, 12)


,문항코드,지문텍스트,보기1,보기2,보기3,보기4,보기5,정답,설명텍스트,난이도,첫번째문장,마지막문장
0,2085,임진왜란과 병자호란 후 조선의 상황으로 알맞지 않은 것은 어느 것입니까?,사람들이 일본에 포로로 끌려갔다.,귀한 문화재들이 불에 타 사라졌다.,많은 군사와 백성들이 죽거나 다쳤다.,농토가 황폐해져 농사짓기가 힘들었다.,나라에서는 세금을 많이 걷을 수 있었다.,⑤,"오랫동안 버려진 농토는 황폐해져 백성들은 농사짓기가 힘들었고, 농사를 짓고 있는 ...",2,임진왜란과 병자호란 후 조선의 상황으로 알맞지 않은 것은 어느 것입니까,임진왜란과 병자호란 후 조선의 상황으로 알맞지 않은 것은 어느 것입니까
1,2086,다음과 같은 농사법으로 농사를 지었을 때의 좋은 점이 아닌 것을 모두 고르시오.\r...,벼가 잘 자란다.,일손이 줄어든다.,잡초가 덜 생긴다.,수확량이 늘어난다.,잡초를 뽑는 횟수가 줄어든다.\r\n⑥ 부농과 빈농의 차이가 없어진다.\r\n⑦ 물...,"⑥,⑦",모내기법의 보급으로 광작을 하는 사람들이 생기자 농민 중에서도 부농과 빈농으로 구...,3,다음과 같은 농사법으로 농사를 지었을 때의 좋은 점이 아닌 것을 모두 고르시오,\r\nㆍ모판에 씨를 뿌려 싹이 난 모를 논에 옮겨 심는 방법입니다
2,2087,조선 시대 농민들이 다음과 같은 작물들을 재배한 까닭은 어느 것입니까?\r\n\r\...,세금이 많아졌기 때문에,양반들이 직접 벼농사를 짓기 시작하였기 때문에,자신이 직접 농사지은 작물만 얻을 수 있었기 때문에,일반 농산물보다 많은 이익을 얻을 수 있었기 때문에,다른 나라에서 새로운 작물을 들여오는 것을 금지하였기 때문에,④,"상품 유통이 활발해지면서 사람들은 장에 내다팔기 위해서 인삼, 담배, 목면, 채소...",3,조선 시대 농민들이 다음과 같은 작물들을 재배한 까닭은 어느 것입니까,조선 시대 농민들이 다음과 같은 작물들을 재배한 까닭은 어느 것입니까
3,2088,조선 후기에 장이 발달하면서 생긴 변화가 아닌 것을 골라 기호를 쓰시오.\r\n\r...,NaN,NaN,NaN,NaN,NaN,㉣,"상인들 중 경강 상인, 송상, 만상, 내상 등은 전국의 장뿐만 아니라 외국에도 물...",3,조선 후기에 장이 발달하면서 생긴 변화가 아닌 것을 골라 기호를 쓰시오,\r\n㉣ 외국의 물건은 들어오지 못하게 되었습니다
4,7197,조선 숙종 때에 이르러 전국적으로 널리 사용된 오른쪽 화폐는 무엇인지 쓰시오.\r\...,NaN,NaN,NaN,NaN,NaN,상평통보,상공업의 발달과 세금의 금납화가 이루어지면서 상평통보는 일차적인 유통 수단이 되어 ...,2,조선 숙종 때에 이르러 전국적으로 널리 사용된 오른쪽 화폐는 무엇인지 쓰시오,조선 숙종 때에 이르러 전국적으로 널리 사용된 오른쪽 화폐는 무엇인지 쓰시오


In [6]:
# 특수문자, 영어 등을 제거 후 Okt로 토큰화

def preprocess_txt(text):
    return re.sub(r'[^ ㄱ-ㅎㅏ-ㅣ가-힣+]+', ' ', text)

okt = Okt()

df['cleaned'] = df['지문텍스트'].apply(preprocess_txt)

corpus = []
for words in df['cleaned']:
    corpus.append(okt.morphs(words))

In [7]:
embedding_model = Word2Vec(corpus, vector_size=20, window = 5, 
                           min_count=1, workers=4, sg=1)

In [8]:
# 각 문항에 대한 평균 벡터 구하기
def get_document_vectors(corpus):
    document_embedding_list = []

    for line in corpus:
        doc2vec = None
        count = 0
        for word in line:
            if any(embedding_model.wv.get_vector(word)):
                count += 1
                if doc2vec is None:
                    doc2vec = embedding_model.wv.get_vector(word)
                else:
                    doc2vec = doc2vec + embedding_model.wv.get_vector(word)

        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(list(doc2vec))

    return document_embedding_list

In [9]:
document_embedding_list = get_document_vectors(corpus)
print(document_embedding_list[:3])

[[0.15250134, 0.48501867, 0.53118765, -0.63673425, 0.12304843, 0.39727548, 0.29402885, 0.5896947, -0.31393582, 0.2641875, 0.098143496, 0.29152814, 0.11789917, -0.23578873, 0.78615296, -0.44704774, 1.6770614, -0.04557995, -1.2998639, -0.10553828], [-0.22006644, 0.18160604, 0.62898237, -0.21163268, -0.10210105, 0.10890714, 0.27807748, 0.5858529, -0.29765993, -0.03212421, 0.3561439, 0.068477884, 0.1401399, -0.33014894, 0.49948144, -0.20789479, 1.0032285, -0.14481983, -0.8614896, -0.15676883], [-0.19697903, 0.23798417, 0.66747224, -0.2528465, 0.029599281, 0.18028478, 0.1386769, 0.72134256, -0.29425624, 0.03037547, 0.13611607, 0.076841414, -0.060993753, -0.46446177, 0.5588821, -0.27269918, 1.2814449, -0.086522445, -0.72309405, -0.0032268497]]


In [14]:
cos_sim = cosine_similarity(document_embedding_list, document_embedding_list)
cos_sim_df = pd.DataFrame(
    data=cos_sim,
    index=df['문항코드'].values,
    columns=df['문항코드'].values
)
cos_sim_df

,2085,2086,2087,2088,7197,7198,7199,7200,7201,7202,...,30075571,30075572,30075574,30075575,30075576,30075577,30075578,30075579,30075580,30075581
2085,1.000000,0.913504,0.929142,0.870158,0.914609,0.991015,0.954176,0.914218,0.986844,0.958590,...,0.843022,0.894328,0.868989,0.871299,0.908550,0.840785,0.832825,0.878632,0.871586,0.927071
2086,0.913504,1.000000,0.962442,0.970976,0.959935,0.921112,0.907926,0.934703,0.917545,0.941876,...,0.930249,0.948177,0.951385,0.938057,0.934299,0.941892,0.945172,0.929335,0.969380,0.974224
2087,0.929142,0.962442,1.000000,0.962623,0.964978,0.935456,0.937908,0.964364,0.929966,0.970215,...,0.912012,0.911173,0.927303,0.905671,0.915486,0.908123,0.925146,0.915421,0.925155,0.956674
2088,0.870158,0.970976,0.962623,1.000000,0.948774,0.885975,0.900680,0.945135,0.888577,0.936149,...,0.940405,0.912500,0.953359,0.936661,0.926062,0.949708,0.959100,0.948245,0.964377,0.963467
7197,0.914609,0.959935,0.964978,0.948774,1.000000,0.911856,0.900019,0.967791,0.905964,0.956114,...,0.908246,0.919654,0.927762,0.933515,0.907672,0.910280,0.906845,0.910563,0.932264,0.946358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30075577,0.840785,0.941892,0.908123,0.949708,0.910280,0.856993,0.846213,0.904361,0.846195,0.863312,...,0.981920,0.926434,0.982680,0.980931,0.949978,1.000000,0.977997,0.951794,0.975179,0.947893
30075578,0.832825,0.945172,0.925146,0.959100,0.906845,0.854987,0.866684,0.907431,0.839026,0.860926,...,0.973210,0.938321,0.977093,0.948383,0.938286,0.977997,1.000000,0.956259,0.980139,0.956985
30075579,0.878632,0.929335,0.915421,0.948245,0.910563,0.878390,0.904534,0.914052,0.870566,0.896722,...,0.955691,0.914174,0.956238,0.954302,0.967491,0.951794,0.956259,1.000000,0.961923,0.956262
30075580,0.871586,0.969380,0.925155,0.964377,0.932264,0.887597,0.891362,0.926920,0.875848,0.894796,...,0.968221,0.959375,0.983750,0.967641,0.952219,0.975179,0.980139,0.961923,1.000000,0.971276


In [15]:
prob_skill = df[['문항코드', 'cleaned']].drop_duplicates()

def get_similar_problem(item_id):

  sim_df = pd.DataFrame(cos_sim_df[item_id].sort_values(ascending=False).reset_index())
  sim_df.columns = ['문항코드', 'similarity']
  sim_df = sim_df[sim_df['문항코드'] != item_id][:10]

  skill_check = prob_skill[prob_skill['문항코드'] == item_id]
  for k in range(len(skill_check)):
    print("Base Question: ",skill_check['cleaned'].iloc[k])

  for i in range(len(sim_df)):
    prob_id = sim_df['문항코드'].iloc[i]
    print("TOP ",i+1, "(",sim_df['similarity'].iloc[i].round(3),") : ", prob_id, "-", end = ' ')
    skill_check = prob_skill[prob_skill['문항코드'] == prob_id]
    for j in range(len(skill_check)):
      print(skill_check['cleaned'].iloc[j], end = ' | ')
    print()
  print()
  
  
def get_different_problem(item_id):

  sim_df = pd.DataFrame(cos_sim_df[item_id].sort_values(ascending=True).reset_index())
  sim_df.columns = ['문항코드', 'similarity']
  sim_df = sim_df[sim_df['문항코드'] != item_id][:10]

  skill_check = prob_skill[prob_skill['문항코드'] == item_id]
  for k in range(len(skill_check)):
    print("Base Question: ",skill_check['cleaned'].iloc[k])

  for i in range(len(sim_df)):
    prob_id = sim_df['문항코드'].iloc[i]
    print("TOP ",i+1, "(",sim_df['similarity'].iloc[i].round(3),") : ", prob_id, "-", end = ' ')
    skill_check = prob_skill[prob_skill['문항코드'] == prob_id]
    for j in range(len(skill_check)):
      print(skill_check['cleaned'].iloc[j], end = ' | ')
    print()
  print()

In [16]:
get_similar_problem(2085)

Base Question:  임진왜란과 병자호란 후 조선의 상황으로 알맞지 않은 것은 어느 것입니까 
TOP  1 ( 0.998 ) :  16434 - 임진왜란과 병자호란 이후의 상황으로 알맞지 않은 것은 어느 것입니까   | 
TOP  2 ( 0.995 ) :  7337 - 임진왜란과 병자호란 이후의 상황으로 알 맞지 않은 것은 어느 것입니까   | 
TOP  3 ( 0.995 ) :  29084 - 임진왜란 때 조선의 상황에 대한 설명으로 바르지 않은 것은 어느 것입니까   | 
TOP  4 ( 0.995 ) :  64837 - 병자호란이 끝난 후에 조선과 청의 관계로 알맞지 않은 것은 어느 것입니까  | 
TOP  5 ( 0.995 ) :  70089 -  세기 조선의 상황으로 알맞지 않은 것은 어느 것입니까  | 
TOP  6 ( 0.994 ) :  30029078 -   운동 이후 변화된 일제의 식민 지배 모습으로 알맞지 않은 것은 어느 것입니까  | 
TOP  7 ( 0.994 ) :  30072672 - 조선의 건국 과정에 대한 설명으로 알맞지 않은 것은 어느 것입니까  | 
TOP  8 ( 0.994 ) :  69545 - 개항 이후 조선의 상황에 대한 설명으로 알맞지 않은 것은 어느 것입니까  | 
TOP  9 ( 0.994 ) :  30049319 - 흥선 대원군의 정책으로 옳지 않은 것은 어느 것입니까  | 
TOP  10 ( 0.993 ) :  55796 - 조선의 건국 과정에 대한 설명으로 옳지 않은 것은 어느 것입니까  | 

